## Import Torch Libraries

In [2]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image


### Load the Car DataSet

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls '/content/drive'

'My Drive'


In [5]:
!cp '/content/drive/My Drive/EVA4Phase2/eva4_phase2_datasets/final_car.zip' .

In [6]:
!unzip -qq final_car.zip

In [7]:
IMAGE_DIM = (128, 128, 3)

### Do Tranformation

In [12]:
transform = transforms.Compose([transforms.Resize((IMAGE_DIM[0],IMAGE_DIM[1])),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5))
                               ]
)

In [18]:
import glob
from PIL import Image
class CARS():
    '''
    CARS Dataset
    You should download this dataset from below url.
    url: https://ai.stanford.edu/~jkrause/cars/car_dataset.html
    '''
    def __init__(self, data_path, transform=None):
        '''
        Args:
            data_path (str): path to dataset
        '''
        self.data_path = data_path
        self.transform = transform
        self.fpaths = sorted(glob.glob(os.path.join(data_path, '*.jpg')))
        #gray_lst = [266, 1085, 2176, 3048, 3439, 3469, 3539, 4577, 4848, 5177, 5502, 5713, 6947, 7383, 7693, 7774, 8137, 8144]
        #for num in gray_lst:
        #    self.fpaths.remove(os.path.join(data_path, '{:05d}.jpg'.format(num)))
        
    def __getitem__(self, idx):
        img = self.transform(Image.open(self.fpaths[idx]))
        return img

    def __len__(self):
        return len(self.fpaths)

In [19]:
train_dataset = CARS(data_path='final_car/train', transform=transform)

In [20]:
test_dataset = CARS(data_path='final_car/test', transform=transform)

### Train and Test Loader

In [23]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, drop_last=True)
train_dl = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, drop_last=True)

In [26]:
sample= next(iter(train_loader))
print(sample.shape)

torch.Size([32, 3, 128, 128])


In [27]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=True, drop_last=True)
test_dl = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=True, drop_last=True)

In [28]:
sample_test= next(iter(test_loader))
print(sample_test.shape)

torch.Size([32, 3, 128, 128])


In [ ]:
CUDA = True
SEED = 1
BATCH_SIZE = 128
LOG_INTERVAL = 10
EPOCHS = 10

# connections through the autoencoder bottleneck
# in the pytorch VAE example, this is 20
ZDIMS = 20



In [ ]:
torch.manual_seed(SEED)
if CUDA:
    torch.cuda.manual_seed(SEED)

# DataLoader instances will load tensors directly into GPU memory
kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}

# Download or load downloaded MNIST dataset
# shuffle data at every epoch


### Define the VAE Class

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # ENCODER
        # 28 x 28 pixels = 784 input pixels, 400 outputs
        self.fc1 = nn.Linear(784, 400)
        # rectified linear unit layer from 400 to 400
        # max(0, x)
        self.relu = nn.ReLU()
        self.fc21 = nn.Linear(400, ZDIMS)  # mu layer
        self.fc22 = nn.Linear(400, ZDIMS)  # logvariance layer
        # this last layer bottlenecks through ZDIMS connections

        # DECODER
        # from bottleneck to hidden 400
        self.fc3 = nn.Linear(ZDIMS, 400)
        # from hidden 400 to 784 outputs
        self.fc4 = nn.Linear(400, 784)
        self.sigmoid = nn.Sigmoid()

    def encode(self, x: Variable) -> (Variable, Variable):
        """Input vector x -> fully connected 1 -> ReLU -> (fully connected
        21, fully connected 22)

        Parameters
        ----------
        x : [128, 784] matrix; 128 digits of 28x28 pixels each

        Returns
        -------

        (mu, logvar) : ZDIMS mean units one for each latent dimension, ZDIMS
            variance units one for each latent dimension

        """

        # h1 is [128, 400]
        h1 = self.relu(self.fc1(x))  # type: Variable
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu: Variable, logvar: Variable) -> Variable:
        """THE REPARAMETERIZATION IDEA:

        For each training sample (we get 128 batched at a time)

        - take the current learned mu, stddev for each of the ZDIMS
          dimensions and draw a random sample from that distribution
        - the whole network is trained so that these randomly drawn
          samples decode to output that looks like the input
        - which will mean that the std, mu will be learned
          *distributions* that correctly encode the inputs
        - due to the additional KLD term (see loss_function() below)
          the distribution will tend to unit Gaussians

        Parameters
        ----------
        mu : [128, ZDIMS] mean matrix
        logvar : [128, ZDIMS] variance matrix

        Returns
        -------

        During training random sample from the learned ZDIMS-dimensional
        normal distribution; during inference its mean.

        """

        if self.training:
            # multiply log variance with 0.5, then in-place exponent
            # yielding the standard deviation
            std = logvar.mul(0.5).exp_()  # type: Variable
            # - std.data is the [128,ZDIMS] tensor that is wrapped by std
            # - so eps is [128,ZDIMS] with all elements drawn from a mean 0
            #   and stddev 1 normal distribution that is 128 samples
            #   of random ZDIMS-float vectors
            eps = Variable(std.data.new(std.size()).normal_())
            # - sample from a normal distribution with standard
            #   deviation = std and mean = mu by multiplying mean 0
            #   stddev 1 sample with desired std and mu, see
            #   https://stats.stackexchange.com/a/16338
            # - so we have 128 sets (the batch) of random ZDIMS-float
            #   vectors sampled from normal distribution with learned
            #   std and mu for the current input
            return eps.mul(std).add_(mu)

        else:
            # During inference, we simply spit out the mean of the
            # learned distribution for the current input.  We could
            # use a random sample from the distribution, but mu of
            # course has the highest probability.
            return mu

    def decode(self, z: Variable) -> Variable:
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x: Variable) -> (Variable, Variable, Variable):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


### Initialize VAE Model

In [ ]:
model = VAE()
if CUDA:
    model.cuda()



### DEfine Loss Function

In [ ]:
def loss_function(recon_x, x, mu, logvar) -> Variable:
    # how well do input x and output recon_x agree?
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784))

    # KLD is Kullback–Leibler divergence -- how much does one learned
    # distribution deviate from another, in this specific case the
    # learned distribution from the unit Gaussian

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # - D_{KL} = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    # note the negative D_{KL} in appendix B of the paper
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # Normalise by same number of elements as in reconstruction
    KLD /= BATCH_SIZE * 784

    # BCE tries to make our reconstruction as accurate as possible
    # KLD tries to push the distributions as close as possible to unit Gaussian
    return BCE + KLD

# Dr Diederik Kingma: as if VAEs weren't enough, he also gave us Adam!
optimizer = optim.Adam(model.parameters(), lr=1e-3)



### Define Train and Test function

In [ ]:
def train(epoch):
    # toggle model to train mode
    model.train()
    train_loss = 0
    # in the case of MNIST, len(train_loader.dataset) is 60000
    # each `data` is of BATCH_SIZE samples and has shape [128, 1, 28, 28]
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        if CUDA:
            data = data.cuda()
        optimizer.zero_grad()

        # push whole batch of data through VAE.forward() to get recon_loss
        recon_batch, mu, logvar = model(data)
        # calculate scalar loss
        loss = loss_function(recon_batch, data, mu, logvar)
        # calculate the gradient of the loss w.r.t. the graph leaves
        # i.e. input variables -- by the power of pytorch!
        loss.backward()
        train_loss += loss.data
        #total_loss += loss_val.data
        #loss_values = [v.data for v in losses]
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


In [ ]:
def test(epoch):
    # toggle model to test / inference mode
    model.eval()
    test_loss = 0

    # each data is of BATCH_SIZE (default 128) samples
    for i, (data, _) in enumerate(test_loader):
        if CUDA:
            # make sure this lives on the GPU
            data = data.cuda()

        # we're only going to infer, so no autograd at all required: volatile=True
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data
        if i == 0:
          n = min(data.size(0), 8)
          # for the first 128 batch of the epoch, show the first 8 input digits
          # with right below them the reconstructed output digits
          comparison = torch.cat([data[:n],
                                  recon_batch.view(BATCH_SIZE, 1, 28, 28)[:n]])
          save_image(comparison.data.cpu(),
                     'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


### Train the Model

In [ ]:
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test(epoch)

    # 64 sets of random ZDIMS-float vectors, i.e. 64 locations / MNIST
    # digits in latent space
    sample = Variable(torch.randn(64, ZDIMS))
    if CUDA:
        sample = sample.cuda()
    sample = model.decode(sample).cpu()

    # save out as an 8x8 matrix of MNIST digits
    # this will give you a visual idea of how well latent space can generate things
    # that look like digits
    save_image(sample.data.view(64, 1, 28, 28),
               'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.001148
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.001178
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.001151
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.001187
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.001115
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.001164
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.001107
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.001160
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.001146
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.001147
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.001151
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.001139
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.001143
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.001119
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.001131
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.001139
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.001162
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.001156
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.001096
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.001140
Train Epoch: 1 [

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  del sys.path[0]


====> Test set loss: 0.0010
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.001063
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.001163
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.001118
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.001081
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.001165
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.001118
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.001110
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.001102
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.001075
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.001134
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.001164
Train Epoch: 2 [14080/60000 (23%)]	Loss: 0.001109
Train Epoch: 2 [15360/60000 (26%)]	Loss: 0.001140
Train Epoch: 2 [16640/60000 (28%)]	Loss: 0.001135
Train Epoch: 2 [17920/60000 (30%)]	Loss: 0.001101
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.001104
Train Epoch: 2 [20480/60000 (34%)]	Loss: 0.001070
Train Epoch: 2 [21760/60000 (36%)]	Loss: 0.001125
Train Epoch: 2 [23040/60000 (38%)]	Loss: 0.001107
Train Epoch: 2 [24320/60000 (41%)]	Los